# Mask R-CNN - Train modified model on old shapes dataset

### the modified model does not include any mask related heads or losses 



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_oldshapes_train, load_model
# model_file = "E:\Models\mrcnn_oldshape_train_logs\TrainMRCNN\mask_rcnn_shapes_1418.h5"
# model_file = "E:\Models\mrcnn_oldshape_train_logs\shapes20180606T1150\mask_rcnn_shapes_0008.h5"
model, dataset_train, dataset_val, train_generator, val_generator, config = prep_oldshapes_train(init_with = 'last', FCN_layers = True, batch_sz = 16)

Using TensorFlow backend.


 windows  Windows
Tensorflow Version: 1.4.0   Keras Version : 2.1.4 
 Initialize config object - super
(56, 56)
>>> Initialize model WITHOUT MASKING LAYERS!!!!
    set_log_dir: Checkpoint path set to : E:\models\mrcnn_oldshape_train_logs\shapes20180613T1232\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     C1 Shape: (?, 32, 32, 64) (?, 32, 32, 64)
     C2 Shape:  (?, 32, 32, 256) (?, 32, 32, 256)
     C3 Shape:  (?, 16, 16, 512) (?, 16, 16, 512)
     C4 Shape:  (?, 8, 8, 1024) (?, 8, 8, 1024)
     C5 Shape:  (?, 4, 4, 2048) (?, 4, 4, 2048)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3 shape : (None, 16, 16, 256)
     FPN P4 shape : (None, 8, 8, 256)
     FPN P5 shape : (None,

TypeError: reduce_max() got an unexpected keyword argument 'keepdims'

In [ ]:
model.layer_info()

In [ ]:
from mrcnn.prep_notebook import load_model
load_model(model, init_with='last')

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

### Training FPN, RPN and MRCNN heads using  Keras.model.fit_generator()

print(config.BATCH_SIZE)
print(model.config.BATCH_SIZE)
print(model.config.LEARNING_RATE)
import keras
keras.__version__

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')
## Last run prior to FCN training was 3699, last checkpoint was 3892  ...start at 3899

train_layers = [ 'mrcnn', 'fpn','rpn']
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
model.epoch = 1233
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 7

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run =3000, 
#             epochs = 25,            
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-6,
            )

###  Train FCN head layers

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')

## Last run prior to FCN training was 3699, last checkpoint was 3892

train_layers = ['fcn']
loss_names   = [  "fcn_norm_loss"]
model.epoch = 1668
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 8 

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run = 500, 
#             epochs = 25,            
#             batch_size = 6,
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-9
            )

In [ ]:
model.keras_model.losses
print(model.keras_model.metrics_names)

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

# train_layers = ['fcn']
# loss_names   = [  "fcn_norm_loss"]
# model.epoch = 208
# model.config.LEARNING_RATE = 1.0e-4
# model.config.STEPS_PER_EPOCH = 8 

# model.train(dataset_train, dataset_val, 
#             learning_rate=model.config.LEARNING_RATE, 
#             epochs_to_run = 500, 
# #             epochs = 25,            
# #             batch_size = 6,
# #             steps_per_epoch = 0 
#             layers = train_layers,
#             losses = loss_names,
#             min_LR = 1.0e-7
#             )

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_post_training.h5")
model.keras_model.save_weights(model_path)

In [ ]:
# model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(model.keras_model, train_batch_x, [ 26], 1)

print(len(model_output))

# rpn_class_loss            = model_output[0]          # layer: 11   shape: (1, 1)
# rpn_bbox_loss             = model_output[1]          # layer: 12   shape: (1, 1)
# mrcnn_class_loss          = model_output[2]          # layer: 13   shape: (1, 1)
# mrcnn_bbox_loss           = model_output[3]          # layer: 14   shape: (1, 1)
# fcn_normalized_loss       = model_output[0]          # layer: 26   shape: (1, 1)

# print(type(output_rois))
for i in model_output:
    print( i.shape)
# print('FCN Normalized Loss is :', fcn_normalized_loss)    


In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)
